In [1]:
import re, random
import spacy 
import numpy as np

In [2]:
# Load the spacy model: nlp
nlp = spacy.load('en_core_web_md')



    Only loading the 'en' tokenizer.



In [3]:
# !python -m spacy download en

In [4]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

print(color.BOLD + 'Hello World !' + color.END)

Hello World !


In [5]:
#Templates to be used throughout 
bot_template = color.BOLD + "BOT" + color.END +": {0}"
user_template =color.BOLD + "USER" + color.END +": {0}"

In [6]:
# Define a function that sends a message to the bot: send_message
def send_message(message):
    # Print user_template including the user_message
    print(user_template.format(message))
    # Get the bot's response to the message
    response = respond(message)
    # Print the bot template including the bot's response.
    print(bot_template.format(response))

## Create dictionary of rules (patterns) to find and associated responses

In [7]:
responses = {'I want (.*)': ['What would it mean if you got {0}',
  'Why do you want {0}',
  "What's stopping you from getting {0}"],
 'do you remember (.*)': ['Did you think I would forget {0}',
  "Why haven't you been able to forget {0}",
  'What about {0}',
  'Yes .. and?'],
 'do you think (.*)': ['if {0}? Absolutely.', 'No chance'],
 'if (.*)': ["Do you really think it's likely that {0}",
  'Do you wish that {0}',
  'What do you think about {0}',
  'Really--if {0}']}

In [8]:
# Define match_rule()
def match_rule(rules, message):
    response, phrase = "default", None
    
    # Iterate over the rules dictionary
    for pattern, responseList in rules.items():
        # Create a match object
        match = re.search(pattern, message)
        if match is not None:
            # Choose a random response
            response = random.choice(responseList)
            if '{0}' in response:
                phrase = match.group(1)
    # Return the response and phrase
    return response, phrase

# Test match_rule
print(match_rule(responses, "do you remember your last birthday"))


('What about {0}', 'your last birthday')


In [9]:
def replace_pronouns(message):

    message = message.lower()
    if 'me' in message:
        # Replace 'me' with 'you'
        return re.sub('me','you',message)
    if 'my' in message:
        # Replace 'my' with 'your'
        return re.sub('my','your',message)
    if 'your' in message:
        # Replace 'your' with 'my'
        return re.sub('your','my',message)
    if 'you' in message:
        # Replace 'you' with 'me'
        return re.sub('you','me',message)

    return message

In [10]:
print(replace_pronouns("my last birthday"))

your last birthday


In [11]:
def respond(message):
    # Call match_rule
    response, phrase = match_rule(responses, message)
    if '{0}' in response:
        # Replace the pronouns in the phrase
        phrase = replace_pronouns(phrase)
        # Include the phrase in the response
        response = response.format(phrase)
    return response

In [12]:
# Send the messages
send_message("do you remember your last birthday")
send_message("do you think humans should be worried about AI")
send_message("I want a robot friend")
send_message("what if you could be anything you wanted")

USER: do you remember your last birthday
BOT: Did you think I would forget my last birthday
USER: do you think humans should be worried about AI
BOT: No chance
USER: I want a robot friend
BOT: What would it mean if you got a robot friend
USER: what if you could be anything you wanted
BOT: Do you wish that me could be anything me wanted


## Intent and Entity Detection

In [13]:
responses.update({'default': 'default message', 'goodbye': 'goodbye for now',\
 'greet': 'Hello you! :)', 'thankyou': 'you are very welcome'})

In [14]:
keywords = {'goodbye': ['bye', 'farewell'],
 'greet': ['hello', 'hi', 'hey'],
 'thankyou': ['thank', 'thx']}

In [15]:
# Define a dictionary of patterns
patterns = {}

# Iterate over the keywords dictionary
for intent, keys in keywords.items():
    # Create regular expressions and compile them into pattern objects
#     patterns[intent] = re.compile('|'.join(keys))    
    patterns[intent] = re.compile("|".join(keys))
    
# Print the patterns
print(patterns)

{'goodbye': re.compile('bye|farewell'), 'greet': re.compile('hello|hi|hey'), 'thankyou': re.compile('thank|thx')}


In [16]:
# Define a function to find the intent of a message
def match_intent(message):
    matched_intent = None
    for intent, pattern in patterns.items():
        # Check if the pattern occurs in the message 
        if pattern.search(message) :
            
            matched_intent = intent
    return matched_intent

In [17]:
# Define a respond function
def respond(message):
    # Call the match_intent function
    intent = match_intent(message)
    print(intent)
    # Fall back to the default response
    key = "default"
    if intent in responses:
        key = intent
    return responses[key]

send_message("hello!")
send_message("bye byeee")
send_message("thanks very much!")

USER: hello!
greet
BOT: Hello you! :)
USER: bye byeee
goodbye
BOT: goodbye for now
USER: thanks very much!
thankyou
BOT: you are very welcome


In [18]:
# Define find_name()
def find_name(message):
    name = None
    # Create a pattern for checking if the keywords occur
    name_keyword = re.compile("name|call")
    # Create a pattern for finding capitalized words
    name_pattern = re.compile("[A-Z]{1}[a-z]*")
    if name_keyword.search(message):
        # Get the matching words in the string
        name_words = name_pattern.findall(message)
        if len(name_words) > 0:
            # Return the name if the keywords are present
            name = ' '.join(name_words)
    return name

In [19]:
# Define respond()
def respond(message):
    # Find the name
    name = find_name(message)
    if name is None:
        return "Hi there!"
    else:
        return "Hello, {0}!".format(name)

# Send messages
send_message("my name is David Copperfield")
send_message("call me Ishmael")
send_message("People call me Cassandra")

USER: my name is David Copperfield
BOT: Hello, David Copperfield!
USER: call me Ishmael
BOT: Hello, Ishmael!
USER: People call me Cassandra
BOT: Hello, People Cassandra!


## Word Vectors

Following data has been extracted from ATIS dataset

In [20]:
sentences = [' i want to fly from boston at 838 am and arrive in denver at 1110 in the morning',
 ' what flights are available from pittsburgh to baltimore on thursday morning',
 ' what is the arrival time in san francisco for the 755 am flight leaving washington',
 ' cheapest airfare from tacoma to orlando',
 ' round trip fares from pittsburgh to philadelphia under 1000 dollars',
 ' i need a flight tomorrow from columbus to minneapolis',
 ' what kind of aircraft is used on a flight from cleveland to dallas',
 ' show me the flights from pittsburgh to los angeles on thursday',
 ' all flights from boston to washington',
 ' what kind of ground transportation is available in denver',
 ' show me the flights from dallas to san francisco',
 ' show me the flights from san diego to newark by way of houston',
 ' what is the cheapest flight from boston to bwi',
 ' all flights to baltimore after 6 pm',
 ' show me the first class fares from boston to denver',
 ' show me the ground transportation in denver',
 ' all flights from denver to pittsburgh leaving after 6 pm and before 7 pm',
 ' i need information on flights for tuesday leaving baltimore for dallas dallas to boston and boston to baltimore',
 ' please give me the flights from boston to pittsburgh on thursday of next week',
 ' i would like to fly from denver to pittsburgh on united airlines',
 ' show me the flights from san diego to newark',
 ' please list all first class flights on united from denver to baltimore',
 ' what kinds of planes are used by american airlines',
 " i'd like to have some information on a ticket from denver to pittsburgh and atlanta",
 " i'd like to book a flight from atlanta to denver",
 ' which airline serves denver pittsburgh and atlanta',
 " show me all flights from boston to pittsburgh on wednesday of next week which leave boston after 2 o'clock pm",
 ' atlanta ground transportation',
 ' i also need service from dallas to boston arriving by noon',
 ' show me the cheapest round trip fare from baltimore to dallas']

In [21]:
labels =['atis_flight', 'atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_airfare', 'atis_flight', 'atis_aircraft',\
 'atis_flight', 'atis_flight', 'atis_ground_service', 'atis_flight', 'atis_flight', 'atis_flight', 'atis_flight',\
         'atis_airfare', 'atis_ground_service', 'atis_flight', 'atis_flight', 'atis_flight', 'atis_flight', 'atis_flight',\
 'atis_flight', 'atis_aircraft', 'atis_airfare', 'atis_flight', 'atis_airline', 'atis_flight', 'atis_ground_service',\
 'atis_flight', 'atis_airfare']

In [22]:
#Calculate the length of sentences using len() and 
n_sentences = len(sentences)

# Calculate the dimensionality of nlp
embedding_dim = nlp.vocab.vectors_length

print(n_sentences, embedding_dim)
# Initialize the array with zeros: X
X = np.zeros((n_sentences, embedding_dim))

# Iterate over the sentences
for idx, sentence in enumerate(sentences):
#     For each sentence, call the nlp object with the sentence as the sole argument. Store the result as doc.
    doc = nlp(sentence)
    
    
    # Save the document's .vector attribute to the corresponding row in X
    X[idx, :] = doc.vector


30 0


ValueError: Word vectors set to length 0. This may be because you don't have a model installed or loaded, or because your model doesn't include word vectors. For more info, see the documentation: 
https://spacy.io/docs/usage


## Entity recognition

In [ ]:
# Define included entities
include_entities = ['DATE', 'ORG', 'PERSON']

# Define extract_entities()
def extract_entities(message):
    # Create a dict to hold the entities
    ents = dict.fromkeys(include_entities)
    # Create a spacy document
    doc = nlp(message)
    for ent in doc.ents:
        if ent.label_ in include_entities:
            # Save interesting entities
            ents[ent.label_] = ent.text
    return ents

print(extract_entities('friends called Mary who have worked at Google since 2010'))
print(extract_entities('people who graduated from MIT in 1999'))


## Negation Detection

In [ ]:
tests = [("no I don't want to be in the south", {'south': False}),
 ('no it should be in the south', {'south': True}),
 ('no in the south not the north', {'north': False, 'south': True}),
 ('not north', {'north': False})]

In [ ]:
def negated_ents(phrase):
    # Extract the entities using keyword matching
    ents = [e for e in ["south", "north"] if e in phrase]
    # Find the index of the final character of each entity
    ends = sorted([phrase.index(e) + len(e) for e in ents])
    # Initialise a list to store sentence chunks
    chunks = []
    # Take slices of the sentence up to and including each entitiy
    start = 0
    for end in ends:
        chunks.append(phrase[start:end])
        start = end
    result = {}
    # Iterate over the chunks and look for entities
    for chunk in chunks:
        for ent in ents:
            if ent in chunk:
                # If the entity is preceeded by a negation, give it the key False
                if "not" in chunk or "n't" in chunk:
                    result[ent] = False
                else:
                    result[ent] = True
    return result 

In [ ]:

# Check that the entities are correctly assigned as True or False
for test in tests:
    print(negated_ents(test[0]) == test[1])
